In [1]:
# pandas and numpy
import pandas as pd
import numpy as np


In [24]:
# Deling with bureau data

In [2]:
# Read in bureau data
bureau = pd.read_csv('bureau.csv')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [8]:
bureau.shape

(1716428, 17)

In [3]:
# Groupby the client id (SK_ID_CURR), count the number of previous loans, and rename the column
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'bureau_prev_loan_counts'})

# Fill the missing values with 0 
previous_loan_counts['bureau_prev_loan_counts'] = previous_loan_counts['bureau_prev_loan_counts'].fillna(0)
previous_loan_counts.head()


,SK_ID_CURR,bureau_prev_loan_counts
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [9]:
previous_loan_counts.shape

(305811, 2)

In [4]:
# Group by the client id and calculate aggregation statistics
bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()

SK_ID_CURR DAYS_CREDIT                               CREDIT_DAY_OVERDUE  \
                   count         mean  max   min   sum              count   
0     100001           7  -735.000000  -49 -1572 -5145                  7   
1     100002           8  -874.000000 -103 -1437 -6992                  8   
2     100003           4 -1400.750000 -606 -2586 -5603                  4   
3     100004           2  -867.000000 -408 -1326 -1734                  2   
4     100005           3  -190.666667  -62  -373  -572                  3   

                 ...    DAYS_CREDIT_UPDATE                               \
  mean max min   ...                 count        mean  max   min   sum   
0  0.0   0   0   ...                     7  -93.142857   -6  -155  -652   
1  0.0   0   0   ...                     8 -499.875000   -7 -1185 -3999   
2  0.0   0   0   ...                     4 -816.000000  -43 -2131 -3264   
3  0.0   0   0   ...                     2 -532.000000 -382  -682 -1064   
4  0.0   0   0   ...                     3  -54.333333  -11  -121  -163   

  AMT_ANNUITY                                      
        count         mean      max  min      sum  
0           7  3545.357143  10822.5  0.0  24817.5  
1           7     0.000000      0.0  0.0      0.0  
2           0          NaN      NaN  NaN      0.0  
3           0          NaN      NaN  NaN      0.0  
4           3  1420.500000   4261.5  0.0   4261.5  

[5 rows x 61 columns]

In [5]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))
            

# Assign the list of columns names as the dataframe column names
bureau_agg.columns = columns
bureau_agg.head()


,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [6]:
# Merge with the training data
bureau_joined = previous_loan_counts.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
bureau_joined.head()

,SK_ID_CURR,bureau_prev_loan_counts,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,7,-735.000000,-49,-1572,-5145,7,0.0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,8,-874.000000,-103,-1437,-6992,8,0.0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,4,-1400.750000,-606,-2586,-5603,4,0.0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,2,-867.000000,-408,-1326,-1734,2,0.0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,3,-190.666667,-62,-373,-572,3,0.0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [7]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,SK_ID_CURR
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354


In [8]:
categorical_grouped = categorical.groupby('SK_ID_CURR' , as_index = False).agg(['sum', 'mean']).reset_index()
categorical_grouped.head()

SK_ID_CURR CREDIT_ACTIVE_Active           CREDIT_ACTIVE_Bad debt       \
                              sum      mean                    sum mean   
0     100001                    3  0.428571                      0  0.0   
1     100002                    2  0.250000                      0  0.0   
2     100003                    1  0.250000                      0  0.0   
3     100004                    0  0.000000                      0  0.0   
4     100005                    2  0.666667                      0  0.0   

  CREDIT_ACTIVE_Closed           CREDIT_ACTIVE_Sold       \
                   sum      mean                sum mean   
0                    4  0.571429                  0  0.0   
1                    6  0.750000                  0  0.0   
2                    3  0.750000                  0  0.0   
3                    2  1.000000                  0  0.0   
4                    1  0.333333                  0  0.0   

  CREDIT_CURRENCY_currency 1 ...  CREDIT_TYPE_Microloan       \
                         sum ...                    sum mean   
0                          7 ...                      0  0.0   
1                          8 ...                      0  0.0   
2                          4 ...                      0  0.0   
3                          2 ...                      0  0.0   
4                          3 ...                      0  0.0   

  CREDIT_TYPE_Mobile operator loan      CREDIT_TYPE_Mortgage       \
                               sum mean                  sum mean   
0                                0  0.0                    0  0.0   
1                                0  0.0                    0  0.0   
2                                0  0.0                    0  0.0   
3                                0  0.0                    0  0.0   
4                                0  0.0                    0  0.0   

  CREDIT_TYPE_Real estate loan      CREDIT_TYPE_Unknown type of loan       
                           sum mean                              sum mean  
0                            0  0.0                                0  0.0  
1                            0  0.0                                0  0.0  
2                            0  0.0                                0  0.0  
3                            0  0.0                                0  0.0  
4                            0  0.0                                0  0.0  

[5 rows x 47 columns]

In [36]:
group_var = 'SK_ID_CURR'

# Need to create new column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in categorical_grouped.columns.levels[0]:
    # Skip the grouping variable
    if var != group_var:
        # Iterate through the stat names
        for stat in ['count', 'count_norm']:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))

#  Rename the columns
categorical_grouped.columns = columns

categorical_grouped.head()

,SK_ID_CURR,CREDIT_ACTIVE_Active_count,CREDIT_ACTIVE_Active_count_norm,CREDIT_ACTIVE_Bad debt_count,CREDIT_ACTIVE_Bad debt_count_norm,CREDIT_ACTIVE_Closed_count,CREDIT_ACTIVE_Closed_count_norm,CREDIT_ACTIVE_Sold_count,CREDIT_ACTIVE_Sold_count_norm,CREDIT_CURRENCY_currency 1_count,...,CREDIT_TYPE_Microloan_count,CREDIT_TYPE_Microloan_count_norm,CREDIT_TYPE_Mobile operator loan_count,CREDIT_TYPE_Mobile operator loan_count_norm,CREDIT_TYPE_Mortgage_count,CREDIT_TYPE_Mortgage_count_norm,CREDIT_TYPE_Real estate loan_count,CREDIT_TYPE_Real estate loan_count_norm,CREDIT_TYPE_Unknown type of loan_count,CREDIT_TYPE_Unknown type of loan_count_norm
0,100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [9]:
bureau_joined_final = bureau_joined.merge(categorical_grouped, on = 'SK_ID_CURR' , how = 'left')
bureau_joined_final.head()
# bureau_joined_final['CREDIT_ACTIVE_Active_count_norm']

C:\Users\bipanigr\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\bipanigr\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:3108: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,SK_ID_CURR,bureau_prev_loan_counts,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,...,"(CREDIT_TYPE_Microloan, sum)","(CREDIT_TYPE_Microloan, mean)","(CREDIT_TYPE_Mobile operator loan, sum)","(CREDIT_TYPE_Mobile operator loan, mean)","(CREDIT_TYPE_Mortgage, sum)","(CREDIT_TYPE_Mortgage, mean)","(CREDIT_TYPE_Real estate loan, sum)","(CREDIT_TYPE_Real estate loan, mean)","(CREDIT_TYPE_Unknown type of loan, sum)","(CREDIT_TYPE_Unknown type of loan, mean)"
0,100001,7,7,-735.000000,-49,-1572,-5145,7,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,100002,8,8,-874.000000,-103,-1437,-6992,8,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,100003,4,4,-1400.750000,-606,-2586,-5603,4,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,100004,2,2,-867.000000,-408,-1326,-1734,2,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,100005,3,3,-190.666667,-62,-373,-572,3,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [10]:
bureau_joined_final.to_csv('bureau_joined_final.csv', index = False)

In [11]:
# Read in bureau balance data
bureau_balance = pd.read_csv('bureau_balance.csv')
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [12]:
categorical1 = pd.get_dummies(bureau_balance.select_dtypes('object'))
categorical1['SK_ID_BUREAU'] = bureau_balance['SK_ID_BUREAU']
categorical1.head()

,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,SK_ID_BUREAU
0,0,0,0,0,0,0,1,0,5715448
1,0,0,0,0,0,0,1,0,5715448
2,0,0,0,0,0,0,1,0,5715448
3,0,0,0,0,0,0,1,0,5715448
4,0,0,0,0,0,0,1,0,5715448


In [14]:
categorical1_grouped = categorical1.groupby('SK_ID_BUREAU' , as_index = False).agg(['sum', 'mean']).reset_index()
categorical1_grouped.head()

SK_ID_BUREAU STATUS_0           STATUS_1      STATUS_2      STATUS_3       \
                    sum      mean      sum mean      sum mean      sum mean   
0      5001709        0  0.000000        0  0.0        0  0.0        0  0.0   
1      5001710        5  0.060241        0  0.0        0  0.0        0  0.0   
2      5001711        3  0.750000        0  0.0        0  0.0        0  0.0   
3      5001712       10  0.526316        0  0.0        0  0.0        0  0.0   
4      5001713        0  0.000000        0  0.0        0  0.0        0  0.0   

  STATUS_4      STATUS_5      STATUS_C           STATUS_X            
       sum mean      sum mean      sum      mean      sum      mean  
0        0  0.0        0  0.0       86  0.886598       11  0.113402  
1        0  0.0        0  0.0       48  0.578313       30  0.361446  
2        0  0.0        0  0.0        0  0.000000        1  0.250000  
3        0  0.0        0  0.0        9  0.473684        0  0.000000  
4        0  0.0        0  0.0        0  0.000000       22  1.000000

In [15]:
columns = ['SK_ID_BUREAU']

# Iterate through the variables names
for var in categorical1_grouped.columns.levels[0]:
    # Skip the grouping variable
    if var != 'SK_ID_BUREAU':
        # Iterate through the stat names
        for stat in ['count', 'count_norm']:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))

#  Rename the columns
categorical1_grouped.columns = columns

categorical1_grouped.head()

,SK_ID_BUREAU,STATUS_0_count,STATUS_0_count_norm,STATUS_1_count,STATUS_1_count_norm,STATUS_2_count,STATUS_2_count_norm,STATUS_3_count,STATUS_3_count_norm,STATUS_4_count,STATUS_4_count_norm,STATUS_5_count,STATUS_5_count_norm,STATUS_C_count,STATUS_C_count_norm,STATUS_X_count,STATUS_X_count_norm
0,5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
1,5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
2,5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
3,5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
4,5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000


In [17]:
# Merge to include the SK_ID_CURR
bureau_by_loan = categorical1_grouped.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')
bureau_by_loan.head()

,SK_ID_BUREAU,STATUS_0_count,STATUS_0_count_norm,STATUS_1_count,STATUS_1_count_norm,STATUS_2_count,STATUS_2_count_norm,STATUS_3_count,STATUS_3_count_norm,STATUS_4_count,STATUS_4_count_norm,STATUS_5_count,STATUS_5_count_norm,STATUS_C_count,STATUS_C_count_norm,STATUS_X_count,STATUS_X_count_norm,SK_ID_CURR
0,5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402,NaN
1,5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446,162368.0
2,5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000,162368.0
3,5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000,162368.0
4,5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000,150635.0


In [18]:
bureau_agg1 = bureau_by_loan.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).sum()
bureau_agg1.head()


,SK_ID_CURR,STATUS_0_count,STATUS_0_count_norm,STATUS_1_count,STATUS_1_count_norm,STATUS_2_count,STATUS_2_count_norm,STATUS_3_count,STATUS_3_count_norm,STATUS_4_count,STATUS_4_count_norm,STATUS_5_count,STATUS_5_count_norm,STATUS_C_count,STATUS_C_count_norm,STATUS_X_count,STATUS_X_count_norm
0,100001.0,31.0,2.356557,1.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,3.088683,30.0,1.502129
1,100002.0,45.0,3.255682,27.0,2.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,1.403409,15.0,1.295455
2,100005.0,14.0,2.205128,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.384615,2.0,0.410256
3,100010.0,20.0,0.555556,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,1.444444,0.0,0.000000
4,100013.0,79.0,1.282872,7.0,0.110804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.0,1.588142,41.0,1.018182


In [19]:
bureau_agg1['SK_ID_CURR'] = bureau_agg1['SK_ID_CURR'].astype(int)
bureau_agg1.head()

,SK_ID_CURR,STATUS_0_count,STATUS_0_count_norm,STATUS_1_count,STATUS_1_count_norm,STATUS_2_count,STATUS_2_count_norm,STATUS_3_count,STATUS_3_count_norm,STATUS_4_count,STATUS_4_count_norm,STATUS_5_count,STATUS_5_count_norm,STATUS_C_count,STATUS_C_count_norm,STATUS_X_count,STATUS_X_count_norm
0,100001,31.0,2.356557,1.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,3.088683,30.0,1.502129
1,100002,45.0,3.255682,27.0,2.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,1.403409,15.0,1.295455
2,100005,14.0,2.205128,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.384615,2.0,0.410256
3,100010,20.0,0.555556,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,1.444444,0.0,0.000000
4,100013,79.0,1.282872,7.0,0.110804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.0,1.588142,41.0,1.018182


In [21]:
bureau_agg1.to_csv('bureau_balance_final.csv', index = False)

In [17]:
app_train = pd.read_csv('application_train.csv')
app_train_joined = app_train.merge(bureau_joined, on = 'SK_ID_CURR', how = 'left')

In [18]:
app_train_joined.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,8.0,-499.875,-7.0,-1185.0,-3999.0,7.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,4.0,-816.000,-43.0,-2131.0,-3264.0,0.0,NaN,NaN,NaN,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,2.0,-532.000,-382.0,-682.0,-1064.0,0.0,NaN,NaN,NaN,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,1.0,-783.000,-783.0,-783.0,-783.0,0.0,NaN,NaN,NaN,0.0
